In [4]:
from astropy.io import ascii, fits
from astropy.coordinates import SkyCoord
from astropy.table import Table, Column, MaskedColumn
from astropy.table import QTable
import astropy.units as u
from astropy.constants import c as c

import json

import os
from glob import glob

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [5]:
with open('C:/Users/kaeka/Documents/Python/SNEcatalog.json', encoding='utf8', errors='ignore') as json_file:
    data = json.load(json_file)

In [6]:
catalog_table = Table()
JSON_keys = data['10Z2-A'].keys()
n = len(data)

catalog_table['ID'] = [i for i in range(n)]

for col_name in JSON_keys:
    catalog_table[col_name] = [None] * n
    
catalog_table['lumdist_kind']  = [None] * n
catalog_table['redshift_kind'] = [None] * n
catalog_table['velocity_kind'] = [None] * n

In [19]:
for i, objname in enumerate(data.keys()):
    record = data[objname]
    
    if len(record['alias']) > 0:
        catalog_table['alias'][i] = ', '.join([h['value'] for h in record['alias']])
        
    if len(record['catalog']) > 0:
        catalog_table['catalog'][i] = record['catalog'][0]
    
    if len(record['claimedtype']) > 0:
        catalog_table['claimedtype'][i] = ', '.join([h['value'] for h in record['claimedtype']])
    
    if len(record['dec']) > 0 and len(record['ra']) > 0:
        coord = SkyCoord(record['ra'][0]['value'], record['dec'][0]['value'], frame ='icrs', unit=('hour','deg'))
        catalog_table['dec'][i] = coord.dec.degree
        catalog_table['ra'][i] = coord.ra.degree

    if len(record['discoverdate']) > 0:
        catalog_table['discoverdate'][i] = ', '.join([h['value'] for h in record['discoverdate']])
        
    if len(record['discoverer']) > 0:
        catalog_table['discoverer'][i] = ', '.join([h['value'] for h in record['discoverer']])
        
    if len(record['download']) > 0:
        catalog_table['download'][i] = record['download'][0]
    
    if len(record['ebv']) > 0:
        catalog_table['ebv'][i] = ', '.join([h['value'] for h in record['ebv']])
    
    if len(record['host']) > 0:
        catalog_table['host'][i] = ', '.join([h['value'] for h in record['host']])

    # Catch "dirty" values that cause an IllegalHourAngle exception.
    try:
        if len(record['hostdec']) > 0 and len(record['hostra']) > 0: 
            h_coord = SkyCoord(record['hostra'][0]['value'], record['hostdec'][0]['value'], frame ='icrs', unit=('hour','deg'))
            catalog_table['hostdec'][i] = h_coord.dec.degree
            catalog_table['hostra'][i] = h_coord.ra.degree
    except:
        catalog_table['hostra'][i] = -999.
        catalog_table['hostdec'][i] = -999.

    if len(record['hostoffsetang']) > 0:
        catalog_table['hostoffsetang'][i] = ', '.join([h['value'] for h in record['hostoffsetang']])
    
    if len(record['hostoffsetdist']) > 0:
        catalog_table['hostoffsetdist'][i] = ', '.join([h['value'] for h in record['hostoffsetdist']])
    
    if len(record['instruments']) > 0:
        catalog_table['instruments'][i] = ', '.join(record['instruments'])
    
    if len(record['lumdist']) > 0:
        lum_ = record['lumdist'][0]
        catalog_table['lumdist'][i] = ', '.join([h['value'] for h in record['lumdist']])
        if 'kind' in lum_:
            lum_kind_ = lum_['kind']
            catalog_table['lumdist_kind'][i] = lum_kind_

    if len(record['maxabsmag']) > 0:
        catalog_table['maxabsmag'][i] = ', '.join([h['value'] for h in record['maxabsmag']])
        
    if len(record['maxappmag']) > 0:
        catalog_table['maxappmag'][i] = ', '.join([h['value'] for h in record['maxappmag']])
        
    if len(record['maxdate']) > 0:
        catalog_table['maxdate'][i] = ', '.join([h['value'] for h in record['maxdate']])
    
    if len(record['name']) > 0:
        catalog_table['name'][i] = record['name'][0]
    
    if len(record['photolink']) > 0:
        catalog_table['photolink'][i] = record['photolink'][0]
        
    if len(record['radiolink']) > 0:
        catalog_table['radiolink'][i] = record['radiolink'][0]
        
    if len(record['redshift']) > 0:
        z_ = record['redshift'][0]
        catalog_table['redshift'][i] = float(record['redshift'][0]['value']) # saving only the first value here
        if 'kind' in z_:
            if isinstance(z_['kind'], list):
                catalog_table['redshift_kind'][i] = ', '.join([h for h in z_['kind']])
            else:
                catalog_table['redshift_kind'][i] = z_['kind']

    if len(record['references']) > 0:
        catalog_table['references'][i] = record['references'][0]

    if len(record['sources']) > 0:
        catalog_table['sources'][i] = record['sources'][0]['name'], record['sources'][0]['url'], record['sources'][0]['alias']

    if len(record['spectralink']) > 0:
        catalog_table['spectralink'][i] = record['spectralink'][0]

    if len(record['velocity']) > 0:
        vel_ = record['velocity'][0]
        catalog_table['velocity'][i] = ', '.join([h['value'] for h in record['velocity']])
        if 'kind' in vel_:
            vel_kind_ = vel_['kind']
            catalog_table['velocity_kind'][i] = vel_kind_

    if len(record['xraylink']) > 0:
        catalog_table['xraylink'][i] = record['xraylink'][0]

In [20]:
# for i, objname in enumerate(data.keys()):
#     record = data[objname]
    
#     if len(record['photolink']) > 0:
#         print(record['photolink'])

In [22]:
catalog_table

ID,alias,catalog,claimedtype,dec,discoverdate,discoverer,download,ebv,host,hostdec,hostoffsetang,hostoffsetdist,hostra,instruments,lumdist,maxabsmag,maxappmag,maxdate,name,photolink,ra,radiolink,redshift,references,sources,spectralink,velocity,xraylink,lumdist_kind,redshift_kind,velocity_kind
int32,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
0,10Z2-A,sne,None,0.011027777777777779,None,None,None,0.0337,None,None,None,None,None,None,None,None,None,None,10Z2-A,None,161.77716666666666,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
1,10Z2-B,sne,None,0.0077777777777777776,None,None,None,0.0383,None,None,None,None,None,None,None,None,None,None,10Z2-B,None,161.42816666666667,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
2,10Z2-D,sne,None,-0.23130555555555557,None,None,None,0.0318,None,None,None,None,None,None,None,None,None,None,10Z2-D,None,161.92058333333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
3,"1156-62, G296.8-00.3, MWSNR 296.8-00.3",sne,None,-62.583333333333336,None,None,None,None,Milky Way,None,None,None,None,None,None,None,None,None,1156-62,None,179.62499999999997,None,None,2014BASI...42...47G,None,None,None,None,None,None,None
4,13Z3-A,sne,None,-0.23947222222222223,None,None,None,0.0241,None,None,None,None,None,None,None,None,None,None,13Z3-A,None,205.87024999999997,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
5,13Z3-C,sne,None,-0.30294444444444446,None,None,None,0.0228,None,None,None,None,None,None,None,None,None,None,13Z3-C,None,206.3599583333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
6,13Z3-D,sne,None,0.11247222222222222,None,None,None,0.0232,None,None,None,None,None,None,None,None,None,None,13Z3-D,None,205.9267083333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
7,13Z3-H,sne,None,0.13444444444444445,None,None,None,0.0238,None,None,None,None,None,None,None,None,None,None,13Z3-H,None,206.36229166666664,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
8,13Z3-I,sne,None,-0.34875,None,None,None,0.022,None,None,None,None,None,None,None,None,None,None,13Z3-I,None,206.07704166666664,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None


In [23]:
# data['psnj2241-2147']['redshift'][0]['kind']

In [24]:
### Checking to see if any elements in the table are lists.
# for col_name in catalog_table.colnames:
#     for i in range(n):
#         if isinstance(catalog_table[col_name][i], list):
#             print(col_name, i, catalog_table[col_name][i])

In [25]:
### Checking for encoding errors, reason being astropy is using ascii encoder, 
### while JSON was read in using utf8 which allows more characters than ascii.
# for col_name in ['alias', 'discoverer', 'name']:
#     for i in range(n):
#         try:
#             catalog_table[col_name][i].encode('ascii')
#         except:
#             if catalog_table[col_name][i] is not None:
#                 print(i, col_name, catalog_table[col_name][i])

In [26]:
catalog_table['discoverer'][36260] = 'K. Sarneczky et al.'
catalog_table['discoverer'][51575] = 'Sarneczky, Kuli, K. Sarneczky et al.'
catalog_table['discoverer'][51672] = 'Kuli, Sarneczky, Z. Kuli, K. Sa'
catalog_table['discoverer'][51697] = 'Sarneczky, Kuli, K. Sarneczky et al.'
catalog_table['discoverer'][52003] = 'Sarneczky, Vinko, Wheeler, K. Sarneczky et al.'
catalog_table['discoverer'][53215] = 'iPTF, K. Sarneczky et al.'
catalog_table['discoverer'][53471] = 'Mederic Boquien, Dalya Baron, Lluis Galbany, Dovi Poznanski'
catalog_table['alias'][57699] = 'SSS120810:231802-560926, SSS120814:221352-124146'
catalog_table['name'][57699] = 'SSS120810:231802-560926'

In [27]:
catalog_table

ID,alias,catalog,claimedtype,dec,discoverdate,discoverer,download,ebv,host,hostdec,hostoffsetang,hostoffsetdist,hostra,instruments,lumdist,maxabsmag,maxappmag,maxdate,name,photolink,ra,radiolink,redshift,references,sources,spectralink,velocity,xraylink,lumdist_kind,redshift_kind,velocity_kind
int32,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
0,10Z2-A,sne,None,0.011027777777777779,None,None,None,0.0337,None,None,None,None,None,None,None,None,None,None,10Z2-A,None,161.77716666666666,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
1,10Z2-B,sne,None,0.0077777777777777776,None,None,None,0.0383,None,None,None,None,None,None,None,None,None,None,10Z2-B,None,161.42816666666667,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
2,10Z2-D,sne,None,-0.23130555555555557,None,None,None,0.0318,None,None,None,None,None,None,None,None,None,None,10Z2-D,None,161.92058333333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
3,"1156-62, G296.8-00.3, MWSNR 296.8-00.3",sne,None,-62.583333333333336,None,None,None,None,Milky Way,None,None,None,None,None,None,None,None,None,1156-62,None,179.62499999999997,None,None,2014BASI...42...47G,None,None,None,None,None,None,None
4,13Z3-A,sne,None,-0.23947222222222223,None,None,None,0.0241,None,None,None,None,None,None,None,None,None,None,13Z3-A,None,205.87024999999997,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
5,13Z3-C,sne,None,-0.30294444444444446,None,None,None,0.0228,None,None,None,None,None,None,None,None,None,None,13Z3-C,None,206.3599583333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
6,13Z3-D,sne,None,0.11247222222222222,None,None,None,0.0232,None,None,None,None,None,None,None,None,None,None,13Z3-D,None,205.9267083333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
7,13Z3-H,sne,None,0.13444444444444445,None,None,None,0.0238,None,None,None,None,None,None,None,None,None,None,13Z3-H,None,206.36229166666664,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
8,13Z3-I,sne,None,-0.34875,None,None,None,0.022,None,None,None,None,None,None,None,None,None,None,13Z3-I,None,206.07704166666664,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None


In [28]:
catalog_table.write('catalog_masterTable.txt', format='ascii.commented_header', overwrite=True)